Meme base et traitement que le notebook de comparaison des Regressors 

In [1]:
import shap
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np 
# import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import  RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.datasets import make_regression
from sklearn.dummy import DummyRegressor
import xgboost as xgb
import multiprocessing
from xgboost import XGBRegressor
from category_encoders import TargetEncoder
from category_encoders import TargetEncoder
import category_encoders as ce
from category_encoders import QuantileEncoder
import time
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from julearn import run_cross_validation
from julearn.utils import configure_logging
from julearn.model_selection import StratifiedGroupsKFold
from julearn.scoring import register_scorer

import math

In [2]:
buildingnr= pd.read_csv("buildingnr.csv",decimal=".",index_col=0)
base_ml = buildingnr

In [3]:
buildingnr['log_SiteEnergyUse(kBtu)_class']= pd.qcut(buildingnr['log_SiteEnergyUse(kBtu)'],4,labels=False)



In [5]:
base_ml = buildingnr[['log_SiteEnergyUse(kBtu)_class','log_SiteEnergyUse(kBtu)',
                     'YearsENERGYSTARCertified', 'ENERGYSTARScore',
    'Area',
 'NaturalGas%',
 'LPU2%',
 'LPU3%',
 'log_PropertyGFATotal',
 'Supermarket/Grocery Store_GFA',
 'Data Center_GFA',
 'Restaurant_GFA',
 'Distribution Center_GFA',
 'Office_GFA%',
 'Office_eui%',
 'Non-Refrigerated Warehouse_eui%',
 'BuildingType',
 'Retail Store_GFA',
 'Non-Refrigerated Warehouse_GFA',
 'Self-Storage Facility_GFA',
 'Office_GFA',
 'Other - Recreation_GFA%',
 'Parking_eui%',
 'Other_GFA',
 'Supermarket/Grocery Store_GFA%',
 'Restaurant_GFA%',
 'Distribution Center_GFA%',
 'Other_GFA%',
 'Retail Store_GFA%',
 'LPU1%',
 'Non-Refrigerated Warehouse_GFA%',
 'Electricity%',
 'ThirdLargestPropertyUseType',
 'PrimaryPropertyType',
 'log_NaturalGas%',
 'log_Electricity%',
 'Latitude',
 'Longitude',
 'NumberofFloors',
 'PropertyGFATotal',
 'PropertyGFABuilding(s)',
 'LargestPropertyUseType',
 'LargestPropertyUseTypeGFA',
 'SecondLargestPropertyUseType',
 'SecondLargestPropertyUseTypeGFA',
 'log_parking%',
 'log_SecondLargestPropertyUseTypeGFA',
 'log_LargestPropertyUseTypeGFA',
 'log_ThirdLargestPropertyUseTypeGFA',
 'log_PropertyGFABuilding(s)',
 'DecBuilt',
 'parking%',
 'LargestUse']]

In [ ]:
base_ml.shape
base_ml.reset_index(inplace = True )
Report = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée']) #initialisation tableau de reporting
Fold_result = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée']) #initialisation tableau de reporting


In [ ]:
colonnes = base_ml.select_dtypes(include= 'object').columns.tolist()
for col in colonnes : 
    mask = base_ml[col].isna()
    base_ml.loc[mask,col]= 'missing'
    
colonnes = base_ml.select_dtypes(include= 'number').columns.tolist()
for col in colonnes : 
    mask = base_ml[col].isna()
    base_ml.loc[mask,col]= 0    

In [ ]:
X = base_ml.drop(['log_SiteEnergyUse(kBtu)','log_SiteEnergyUse(kBtu)_class'],axis=1)
y = base_ml['log_SiteEnergyUse(kBtu)']
X.columns

In [ ]:
colQE= base_ml.select_dtypes(include= 'object').columns.tolist()
colQE.append('DecBuilt')

enc = QuantileEncoder(cols=colQE, quantile=0.5, m=0).fit(X, y)
X_QE= enc.transform(X)


X_QE=enc.transform(X)
X=X_QE.drop(columns='index')





Faire varier les différents hyperparametres pour déterminer les plus efficaces et analyses graphiques

In [ ]:
debut=  time.time()
nsplit =5
param_courbe = []
R2 = []
courbe_param =pd.DataFrame(columns=['Parametre', 'R2 test']) 
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])  

for i in range(2,25,1) : 
    
    param = [i]
    param_courbe.append(i)
    
    tuned_parameters = { 'max_depth': [7], 'min_samples_split': param, 'n_estimators' : [100]}
    
   
    
    reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                          ('regression', GridSearchCV(RandomForestRegressor(random_state=22), tuned_parameters, 
                              cv=2, n_jobs=-4, verbose=0))]) 
                      

    skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 )
    Fold_result = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée']) 
 
    for train_index, test_index in skf.split(X, buildingnr['log_SiteEnergyUse(kBtu)_class']):
        x_train_fold, x_test_fold = X.iloc[train_index] , X.iloc[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        reg.fit(x_train_fold, y_train_fold)
  
    
        Fold_result_temp = pd.DataFrame([['RandomForest', 
                          np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold)),3),
                           np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold),squared=False),3),
                          np.around(r2_score(y_test_fold,reg.predict(x_test_fold)),3),
                         np.around( mean_absolute_percentage_error(y_test_fold,reg.predict(x_test_fold)),3),
                          np.around(r2_score(y_train_fold,reg.predict(x_train_fold)),3) ,
                          time.time() - debut]], 
                         columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
        Fold_result = pd.concat([Fold_result, Fold_result_temp])
        pass


    Append = pd.DataFrame([['RandomForest', 
                        np.around (Fold_result['MSE'].mean(),3),
                       np.around(Fold_result['RMSE'].mean(),3),
                      np.around(Fold_result['R2 Test'].mean(),3),
                     np.around( Fold_result['MA%E'].mean(),3),
                      np.around(Fold_result['R2 Train'].mean(),3) ,
                      np.around(Fold_result['Durée'].mean(),3) ]], 
                      columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
    Report = pd.concat([Report, Append]) 
    append_param=pd.DataFrame([[i,Fold_result['R2 Test'].mean()]],columns=['Parametre', 'R2 test'])
    courbe_param = pd.concat([courbe_param,append_param])

    
         
    R2.append(Fold_result['R2 Test'].mean())


    
      
print ( Fold_result)
courbe_param

In [ ]:
ax = plt.gca()

ax.plot(param_courbe, R2)
#ax.set_xscale('log')
plt.xlabel('Parametre min sample split')
plt.ylabel('R2')
plt.axis('tight')
plt.show()
print ("Meilleur R2 " , max(R2)) 
print (param_courbe)

In [ ]:
nsplit =5
param_courbe = []
R2 = []
courbe_param =pd.DataFrame(columns=['Parametre', 'R2 test']) 
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])  

for i in range(2,10,1) : 
    
    param = [i]
    param_courbe.append(i)
    
    tuned_parameters = {'max_depth': [7], 'min_samples_split': [3],'n_estimators' : [100]}

    reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                          ('regression', GridSearchCV(RandomForestRegressor(random_state=22), tuned_parameters, 
                                        cv=i, n_jobs=-4, verbose=1))]) 
                      

    skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 )
    Fold_result = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée']) 
 
    for train_index, test_index in skf.split(X, buildingnr['log_SiteEnergyUse(kBtu)_class']):
        x_train_fold, x_test_fold = X.iloc[train_index] , X.iloc[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        reg.fit(x_train_fold, y_train_fold)
  
    
        Fold_result_temp = pd.DataFrame([['RandomForest', 
                          np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold)),3),
                           np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold),squared=False),3),
                          np.around(r2_score(y_test_fold,reg.predict(x_test_fold)),3),
                         np.around( mean_absolute_percentage_error(y_test_fold,reg.predict(x_test_fold)),3),
                          np.around(r2_score(y_train_fold,reg.predict(x_train_fold)),3) ,
                          time.time() - debut]], 
                         columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
        Fold_result = pd.concat([Fold_result, Fold_result_temp])
        pass


    Append = pd.DataFrame([['RandomForest', 
                        np.around (Fold_result['MSE'].mean(),3),
                       np.around(Fold_result['RMSE'].mean(),3),
                      np.around(Fold_result['R2 Test'].mean(),3),
                     np.around( Fold_result['MA%E'].mean(),3),
                      np.around(Fold_result['R2 Train'].mean(),3) ,
                      np.around(Fold_result['Durée'].mean(),3) ]], 
                      columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
    Report = pd.concat([Report, Append]) 
    append_param=pd.DataFrame([[i,Fold_result['R2 Test'].mean()]],columns=['Parametre', 'R2 test'])
    courbe_param = pd.concat([courbe_param,append_param])
    
    
    R2.append(Fold_result['R2 Test'].mean())
      
    Fold_result

print ( Fold_result)
courbe_param

In [ ]:
ax = plt.gca()

ax.plot(param_courbe, R2)
#ax.set_xscale('log')
plt.xlabel('Parametre nb CV')
plt.ylabel('R2')
plt.axis('tight')
plt.show()
print ("Meilleur R2 " , max(R2))

In [ ]:
nsplit =5
param_courbe = []
R2 = []
courbe_param =pd.DataFrame(columns=['Parametre', 'R2 test']) 
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])  

for i in range(1,20,1) : 
    
    param = [i]
    param_courbe.append(i)
    
    tuned_parameters = {'max_depth': [i], 'min_samples_split': [3], 'min_samples_leaf':[3],'n_estimators' : [100]}

    reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                          ('regression', GridSearchCV(RandomForestRegressor(random_state=22), tuned_parameters, 
                                        cv=2, n_jobs=-4, verbose=1))]) 
                      

    skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 )
    Fold_result = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée']) 
 
    for train_index, test_index in skf.split(X, buildingnr['log_SiteEnergyUse(kBtu)_class']):
        x_train_fold, x_test_fold = X.iloc[train_index] , X.iloc[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        reg.fit(x_train_fold, y_train_fold)
  
    
        Fold_result_temp = pd.DataFrame([['RandomForest', 
                          np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold)),3),
                           np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold),squared=False),3),
                          np.around(r2_score(y_test_fold,reg.predict(x_test_fold)),3),
                         np.around( mean_absolute_percentage_error(y_test_fold,reg.predict(x_test_fold)),3),
                          np.around(r2_score(y_train_fold,reg.predict(x_train_fold)),3) ,
                          time.time() - debut]], 
                         columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
        Fold_result = pd.concat([Fold_result, Fold_result_temp])
        pass


    Append = pd.DataFrame([['RandomForest', 
                        np.around (Fold_result['MSE'].mean(),3),
                       np.around(Fold_result['RMSE'].mean(),3),
                      np.around(Fold_result['R2 Test'].mean(),3),
                     np.around( Fold_result['MA%E'].mean(),3),
                      np.around(Fold_result['R2 Train'].mean(),3) ,
                 np.around(Fold_result['Durée'].mean(),3) ]], 
                      columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
    Report = pd.concat([Report, Append]) 
    append_param=pd.DataFrame([[i,Fold_result['R2 Test'].mean()]],columns=['Parametre', 'R2 test'])
    courbe_param = pd.concat([courbe_param,append_param])
    
    
    R2.append(Fold_result['R2 Test'].mean())
      
    Fold_result

courbe_param=courbe_param.reset_index()
courbe_param=courbe_param.drop(columns = 'index')
print ( Fold_result)
courbe_param

In [ ]:
sns.lineplot(data = courbe_param, x='Parametre',y='R2 test').set(
    title="RF  - max_depth")

In [ ]:
nsplit =5
param_courbe = []
R2 = []
courbe_param =pd.DataFrame(columns=['Parametre', 'R2 test']) 
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])  

for i in range(1,20,1) : 
    
    param = [i]
    param_courbe.append(i)
    
    tuned_parameters = { 'min_samples_leaf':[i],'max_depth': [7], 'min_samples_split': [3], 'n_estimators' : [100]}

    reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                          ('regression', GridSearchCV(RandomForestRegressor(random_state=22),
                                            tuned_parameters,  cv=2, n_jobs=-4, verbose=1))]) 
    
 
    
                      

    skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 )
    Fold_result = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée']) 
 
    for train_index, test_index in skf.split(X, buildingnr['log_SiteEnergyUse(kBtu)_class']):
        x_train_fold, x_test_fold = X.iloc[train_index] , X.iloc[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        reg.fit(x_train_fold, y_train_fold)
  
    
        Fold_result_temp = pd.DataFrame([['RandomForest', 
                        np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold)),3),
                        np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold),squared=False),3),
                        np.around(r2_score(y_test_fold,reg.predict(x_test_fold)),3),
                        np.around( mean_absolute_percentage_error(y_test_fold,reg.predict(x_test_fold)),3),
                        np.around(r2_score(y_train_fold,reg.predict(x_train_fold)),3) ,
                        time.time() - debut]], 
                        columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
        Fold_result = pd.concat([Fold_result, Fold_result_temp])
        pass


    Append = pd.DataFrame([['RandomForest', 
                        np.around (Fold_result['MSE'].mean(),3),
                        np.around(Fold_result['RMSE'].mean(),3),
                        np.around(Fold_result['R2 Test'].mean(),3),
                        np.around( Fold_result['MA%E'].mean(),3),
                        np.around(Fold_result['R2 Train'].mean(),3) ,
                        np.around(Fold_result['Durée'].mean(),3) ]], 
                        columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
    Report = pd.concat([Report, Append]) 
    append_param=pd.DataFrame([[i,Fold_result['R2 Test'].mean()]],columns=['Parametre', 'R2 test'])
    courbe_param = pd.concat([courbe_param,append_param])
    
    
    R2.append(Fold_result['R2 Test'].mean())
      
    Fold_result

courbe_param=courbe_param.reset_index()
courbe_param=courbe_param.drop(columns = 'index')
print ( Fold_result)
courbe_param

In [ ]:
sns.lineplot(data = courbe_param, x='Parametre',y='R2 test').set(
    title="RF  - min sample leaf")

In [ ]:
debut=  time.time()
nsplit =5
param_courbe = []
R2 = []
courbe_param =pd.DataFrame(columns=['Parametre', 'R2 test']) 
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])  

for i in range(1,500,100) : 
    
    param = [i]
    param_courbe.append(i)
    
    tuned_parameters = { 'min_samples_leaf':[1],'max_depth': [7], 'min_samples_split': [3], 'n_estimators' : [i]}

    reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                          ('regression', GridSearchCV(RandomForestRegressor(random_state=22),
                                            tuned_parameters,  cv=2, n_jobs=-4, verbose=1))]) 
    
 
    
                      

    skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 )
    Fold_result = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée']) 
 
    for train_index, test_index in skf.split(X, buildingnr['log_SiteEnergyUse(kBtu)_class']):
        x_train_fold, x_test_fold = X.iloc[train_index] , X.iloc[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        reg.fit(x_train_fold, y_train_fold)
  
    
        Fold_result_temp = pd.DataFrame([['RandomForest', 
                        np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold)),3),
                        np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold),squared=False),3),
                        np.around(r2_score(y_test_fold,reg.predict(x_test_fold)),3),
                        np.around( mean_absolute_percentage_error(y_test_fold,reg.predict(x_test_fold)),3),
                        np.around(r2_score(y_train_fold,reg.predict(x_train_fold)),3) ,
                        time.time() - debut]], 
                        columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
        Fold_result = pd.concat([Fold_result, Fold_result_temp])
        pass


    Append = pd.DataFrame([['RandomForest', 
                        np.around (Fold_result['MSE'].mean(),3),
                        np.around(Fold_result['RMSE'].mean(),3),
                        np.around(Fold_result['R2 Test'].mean(),3),
                        np.around( Fold_result['MA%E'].mean(),3),
                        np.around(Fold_result['R2 Train'].mean(),3) ,
                        np.around(Fold_result['Durée'].mean(),3) ]], 
                        columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
    Report = pd.concat([Report, Append]) 
    append_param=pd.DataFrame([[i,Fold_result['R2 Test'].mean()]],columns=['Parametre', 'R2 test'])
    courbe_param = pd.concat([courbe_param,append_param])
    
    

      


Fold_result


In [ ]:
courbe_param=courbe_param.reset_index()
courbe_param=courbe_param.drop(columns = 'index')
courbe_param

In [ ]:
sns.lineplot(data = courbe_param, x='Parametre',y='R2 test').set(
    title="RF  - n_estimator")

In [ ]:
nsplit =5
param_courbe = []
R2 = []
courbe_param =pd.DataFrame(columns=['Parametre', 'R2 test']) 
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])  

for i in range(0,100,10):

    enc = QuantileEncoder(cols=base_ml.select_dtypes(include= 'object').columns,quantile=i/100, m=1.0).fit(X, y) 
    X_QE= enc.transform(X)
    X=X_QE
    
    param = [i/100]
    param_courbe.append(i/100)
    
    
    tuned_parameters = { 'min_samples_leaf':[1],'max_depth': [7], 'min_samples_split': [3],'n_estimators' : [100]}

    reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                          ('regression', GridSearchCV(RandomForestRegressor(random_state=22),
                                            tuned_parameters,  cv=2, n_jobs=-4, verbose=1))]) 
    
 
    
                      

    skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 )
    Fold_result = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée']) 
 
    for train_index, test_index in skf.split(X, buildingnr['log_SiteEnergyUse(kBtu)_class']):
        x_train_fold, x_test_fold = X.iloc[train_index] , X.iloc[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        reg.fit(x_train_fold, y_train_fold)
  
    
        Fold_result_temp = pd.DataFrame([['RandomForest', 
                        np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold)),3),
                        np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold),squared=False),3),
                        np.around(r2_score(y_test_fold,reg.predict(x_test_fold)),3),
                        np.around( mean_absolute_percentage_error(y_test_fold,reg.predict(x_test_fold)),3),
                        np.around(r2_score(y_train_fold,reg.predict(x_train_fold)),3) ,
                        time.time() - debut]], 
                        columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
        Fold_result = pd.concat([Fold_result, Fold_result_temp])
        pass


    Append = pd.DataFrame([['RandomForest', 
                        np.around (Fold_result['MSE'].mean(),3),
                        np.around(Fold_result['RMSE'].mean(),3),
                        np.around(Fold_result['R2 Test'].mean(),3),
                        np.around( Fold_result['MA%E'].mean(),3),
                        np.around(Fold_result['R2 Train'].mean(),3) ,
                        np.around(Fold_result['Durée'].mean(),3) ]], 
                        columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
    Report = pd.concat([Report, Append]) 
    append_param=pd.DataFrame([[i,Fold_result['R2 Test'].mean()]],columns=['Parametre', 'R2 test'])
    courbe_param = pd.concat([courbe_param,append_param])
    
    
    R2.append(Fold_result['R2 Test'].mean())
      
    print (Fold_result)

courbe_param=courbe_param.reset_index()
courbe_param=courbe_param.drop(columns = 'index')


In [ ]:
courbe_param=courbe_param.reset_index()
courbe_param=courbe_param.drop(columns = 'index')
courbe_param

In [ ]:
sns.lineplot(data = courbe_param, x='Parametre',y='R2 test').set(
    title="RF  - Quantile encoder : quantile")

In [ ]:
nsplit =5
param_courbe = []
R2 = []
courbe_param =pd.DataFrame(columns=['Parametre', 'R2 test']) 
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])  

for i in range(0,100,10):

    enc = QuantileEncoder(cols=base_ml.select_dtypes(include= 'object').columns,quantile=0.1, m=i/100).fit(X, y) 
    X_QE= enc.transform(X)
    X=X_QE
    
    param = [i/100]
    param_courbe.append(i/100)
    
    
    tuned_parameters = { 'min_samples_leaf':[3],'max_depth': [7], 'min_samples_split': [5],'n_estimators' : [250]}

    reg = Pipeline(steps=[('preprocessor', numeric_transformer),
                          ('regression', GridSearchCV(RandomForestRegressor(random_state=22),
                                            tuned_parameters,  cv=2, n_jobs=-4, verbose=1))]) 
                   
    skf = StratifiedKFold(n_splits=nsplit, shuffle=True,random_state=22 )
    Fold_result = pd.DataFrame(columns=['méthode', 'MSE','RMSE','R2 Test',"MA%E",'R2 Train','Durée']) 
 
    for train_index, test_index in skf.split(X, buildingnr['log_SiteEnergyUse(kBtu)_class']):
        x_train_fold, x_test_fold = X.iloc[train_index] , X.iloc[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        reg.fit(x_train_fold, y_train_fold)
  
    
        Fold_result_temp = pd.DataFrame([['RandomForest', 
                        np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold)),3),
                        np.around (mean_squared_error(y_test_fold,reg.predict(x_test_fold),squared=False),3),
                        np.around(r2_score(y_test_fold,reg.predict(x_test_fold)),3),
                        np.around( mean_absolute_percentage_error(y_test_fold,reg.predict(x_test_fold)),3),
                        np.around(r2_score(y_train_fold,reg.predict(x_train_fold)),3) ,
                        time.time() - debut]], 
                        columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
        Fold_result = pd.concat([Fold_result, Fold_result_temp])
        pass


    Append = pd.DataFrame([['RandomForest', 
                        np.around (Fold_result['MSE'].mean(),3),
                        np.around(Fold_result['RMSE'].mean(),3),
                        np.around(Fold_result['R2 Test'].mean(),3),
                        np.around( Fold_result['MA%E'].mean(),3),
                        np.around(Fold_result['R2 Train'].mean(),3) ,
                        np.around(Fold_result['Durée'].mean(),3) ]], 
                        columns=['méthode', 'MSE','RMSE','R2 Test','MA%E','R2 Train','Durée'])
    Report = pd.concat([Report, Append]) 
    append_param=pd.DataFrame([[i,Fold_result['R2 Test'].mean()]],columns=['Parametre', 'R2 test'])
    courbe_param = pd.concat([courbe_param,append_param])
    
    
    R2.append(Fold_result['R2 Test'].mean())
      
    Fold_result

courbe_param=courbe_param.reset_index()
courbe_param=courbe_param.drop(columns = 'index')
courbe_param

In [ ]:
courbe_param=courbe_param.reset_index()
courbe_param=courbe_param.drop(columns = 'index')
sns.lineplot(data = courbe_param, x='Parametre',y='R2 test').set(
    title="RF  - Quantile encoder : m")

Analyse des Features importance avec la libray Shap

In [ ]:
col_names=base_ml.drop(columns = 'DecBuilt').select_dtypes(include='number').columns.tolist()
features = base_ml[col_names]
scaler = StandardScaler().fit(features.values)
features_scale = scaler.transform(features.values)
base_ml[col_names] = features_scale


In [ ]:
colQE= base_ml.select_dtypes(include= 'object').columns.tolist()
colQE.append('DecBuilt')

enc = QuantileEncoder(cols=colQE, quantile=0.1, m=0.5).fit(X, y)
X_QE= enc.transform(X)



In [ ]:
reg = RandomForestRegressor(random_state=22,max_depth =7, min_samples_split= 3, min_samples_leaf=1,n_estimators=100)
skf = StratifiedKFold(n_splits=5, shuffle=True,random_state=22 )
for train_index, test_index in skf.split(X, buildingnr['log_SiteEnergyUse(kBtu)_class']):
    x_train_fold, x_test_fold = X.iloc[train_index] , X.iloc[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    reg.fit(x_train_fold, y_train_fold)

In [ ]:
import shap
shap_values = shap.TreeExplainer(reg).shap_values(x_train_fold)
shap.summary_plot(shap_values, x_train_fold, plot_type="bar")

In [ ]:


explainer = shap.Explainer(reg.predict, x_test_fold)
# Calculates the SHAP values - It takes some time
shap_values = explainer(x_test_fold)



In [ ]:
from scipy.special import softmax

def print_feature_importances_shap_values(shap_values, features):
    '''
    Prints the feature importances based on SHAP values in an ordered way
    shap_values -> The SHAP values calculated from a shap.Explainer object
    features -> The name of the features, on the order presented to the explainer
    '''
    # Calculates the feature importance (mean absolute shap value) for each feature
    importances = []
    for i in range(shap_values.values.shape[1]):
        importances.append(np.mean(np.abs(shap_values.values[:, i])))
    # Calculates the normalized version
    importances_norm = softmax(importances)
    # Organize the importances and columns in a dictionary
    feature_importances = {fea: imp for imp, fea in zip(importances, features)}
    feature_importances_norm = {fea: imp for imp, fea in zip(importances_norm, features)}
    # Sorts the dictionary
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}
    feature_importances_norm= {k: v for k, v in sorted(feature_importances_norm.items(), key=lambda item: item[1], reverse = True)}
    # Prints the feature importances
    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f} (softmax = {feature_importances_norm[k]:.4f})")

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.summary_plot(shap_values)

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.summary_plot(shap_values, plot_type='violin')

In [ ]:
shap.plots.bar(shap_values[0])

In [ ]:
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.plots.force(shap_values[0])